In [3]:
import cv2
import numpy as np

src1 = cv2.imread("image1_1.jpg")
src1 = cv2.resize(src1, None, fx=0.3, fy=0.3)


In [4]:
# 創建一個圖像副本，用來畫出輪廓，不影響原始圖像
image_with_rect = src1.copy()
# 創建一個空白遮罩
mask = np.zeros(src1.shape[:2], dtype=np.uint8)
# 將圖像轉為灰階
gray = cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
# 進行二值化，將物體變為白色，背景變為黑色
_, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
# 尋找輪廓
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

if contours:
    # 找到面積最大的輪廓
    largest_contour = max(contours, key=cv2.contourArea)
    
    # 計算輪廓的周長
    perimeter = cv2.arcLength(largest_contour, True)
    
    # 使用 approxPolyDP 簡化輪廓，嘗試找到四個頂點
    epsilon = 0.04 * perimeter
    approx_poly = cv2.approxPolyDP(largest_contour, epsilon, True)
    
    # 確保我們找到的是一個四邊形
    if len(approx_poly) == 4:
        # 將 numpy 陣列重塑為 (4, 2)
        box = approx_poly.reshape(4, 2)
        
        # ------------------- 關鍵步驟：對頂點進行排序 -------------------
        
        # 創建一個空的 numpy 陣列來存放排序後的頂點
        sorted_points = np.zeros((4, 2), dtype=np.int32)
        
        # 1. 根據 x + y 的總和排序，找到左上角和右下角的點
        s = box.sum(axis=1)
        sorted_points[0] = box[np.argmin(s)]  # 總和最小的是左上角
        sorted_points[2] = box[np.argmax(s)]  # 總和最大的是右下角
        
        # 2. 根據 x - y 的差值排序，找到右上角和左下角的點
        d = np.diff(box, axis=1)
        sorted_points[1] = box[np.argmin(d)]  # 差值最小的是左下角
        sorted_points[3] = box[np.argmax(d)]  # 差值最大的是右上角
        
        # 注意：上面的排序邏輯可能會因圖像和角度而有小幅變動，另一種更可靠的排序方式是
        # 根據頂點和中心點的角度進行排序。但對於大多數情況，這個方法足夠了。

        # ------------------- 繪製並應用遮罩 -------------------
        
        # 建立一個黑色遮罩
        mask = np.zeros(src1.shape[:2], dtype=np.uint8)
        
        # 使用排序後的頂點繪製並填充四邊形
        # 注意：這裡直接使用 sorted_points，不需要再用 minAreaRect
        # 因為 approxPolyDP 的點更接近輪廓的實際頂點
        cv2.fillPoly(mask, [sorted_points], 255)
        
        # 應用遮罩
        result = cv2.bitwise_and(src1, src1, mask=mask)
        
        # 顯示結果
        cv2.imshow('Original Image', src1)
        cv2.imshow('Binary Mask', binary)
        cv2.imshow('Extracted Area with Corrected Quadrilateral', result)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        print("簡化後的輪廓不是一個四邊形。請嘗試調整 epsilon。")
else:
    print("沒有找到任何輪廓。")